# Sentiment analysis

In [69]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [70]:
# load the IMDb data - already preprocessed! numpy arrays, consisting of lists
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [71]:
type(X_train[0])

list

The words are indexed by frequency; 0, 1, and 2 are special:
* 0 is padding token
* 1 is start-of-sequence (SSS) token
* 2 are unknown words

Decoding works as follows:

In [72]:
word_index = keras.datasets.imdb.get_word_index()

In [73]:
# the word_index and the encoded sentences are shifted by 3 positions (because of the above 3 special characters)
word_index['this'] # this word is number 14 in encoded sentences -- one of them shown above

11

In [74]:
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token

In [75]:
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

## Custom preprocessing

In [66]:
import tensorflow_datasets as tfds

In [76]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples

In [78]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

In [79]:
# construct the vocabulary
from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [80]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [81]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [82]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [87]:
table.lookup(tf.constant([b"a e i o u".split()]))

<tf.Tensor: shape=(1, 5), dtype=int64, numpy=array([[   2,  702,   71,  504, 1589]])>

In [84]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch
train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [89]:
# create the model and train it

embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape = [None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

In [90]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 70s 89ms/step - loss: 0.6478 - accuracy: 0.5995
Epoch 2/5
782/782 [==============================] - 74s 94ms/step - loss: 0.4631 - accuracy: 0.7824
Epoch 3/5
782/782 [==============================] - 62s 79ms/step - loss: 0.3072 - accuracy: 0.8734
Epoch 4/5
782/782 [==============================] - 84s 108ms/step - loss: 0.1631 - accuracy: 0.9442
Epoch 5/5
782/782 [==============================] - 60s 77ms/step - loss: 0.1150 - accuracy: 0.9609


## Masking

Important: this assumes that the padding token is encoded as 0 (or a vector of all zeros).
Masking simply tells the RNN that the input at a given 'timestep' is a padding token and it 
should not be paid attention to (usually, RNN layers simply ignore it and just copy the output from the previous timestep). Interestingly, masking in the example below does not help performace at all.

In [92]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape = [None], 
                           mask_zero = True),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

In [93]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 61s 78ms/step - loss: 0.5345 - accuracy: 0.7219
Epoch 2/5
782/782 [==============================] - 75s 95ms/step - loss: 0.3333 - accuracy: 0.8610
Epoch 3/5
782/782 [==============================] - 61s 78ms/step - loss: 0.1814 - accuracy: 0.9359
Epoch 4/5
782/782 [==============================] - 62s 79ms/step - loss: 0.1328 - accuracy: 0.9521
Epoch 5/5
782/782 [==============================] - 59s 76ms/step - loss: 0.1114 - accuracy: 0.9601
